In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()
import gc

In [2]:
%%time
training_data_df = pd.read_csv('./Prepped Data/training_data7_one_hot_encoding_20220207.csv')
print(training_data_df.shape)
training_data_df.head()

(2091509, 458)
Wall time: 40.8 s


,주소_건축년도,건축년도,전용면적(㎡),층,x좌표,y좌표,계약날짜일수_2006년기준,계약날짜기준_건물연식,계약날짜일수_sin,계약날짜일수_cos,거래/보증금(만원),부동산유형_아파트,부동산유형_오피스텔,부동산유형_연립다세대,브랜드_기타,브랜드_현대,브랜드_삼성,브랜드_한신,브랜드_래미안,브랜드_아이파크,브랜드_편한세상,브랜드_아펠바움,브랜드_아크로,브랜드_롯데캐슬,브랜드_더샵,브랜드_코오롱,브랜드_SK,브랜드_힐스테이트,브랜드_푸르지오,브랜드_레미안,브랜드_LG,브랜드_데시앙,브랜드_써밋,브랜드_디에이치,브랜드_리슈,브랜드_대방,브랜드_호반,브랜드_포레나,브랜드_중흥,구_강남구,구_강동구,구_강북구,구_강서구,구_관악구,구_광진구,구_구로구,구_금천구,구_노원구,구_도봉구,구_동대문구,구_동작구,구_마포구,구_서대문구,구_서초구,구_성동구,구_성북구,구_송파구,구_양천구,구_영등포구,구_용산구,구_은평구,구_종로구,구_중구,구_중랑구,동_개포동,동_논현동,동_대치동,동_도곡동,동_삼성동,동_세곡동,동_수서동,동_신사동,동_압구정동,동_역삼동,동_율현동,동_일원동,동_자곡동,동_청담동,동_강일동,동_고덕동,동_길동,동_둔촌동,동_명일동,동_상일동,동_성내동,동_암사동,동_천호동,동_미아동,동_번동,동_수유동,동_우이동,동_가양동,동_개화동,동_공항동,동_내발산동,동_등촌동,동_마곡동,동_방화동,동_염창동,동_외발산동,동_화곡동,동_남현동,동_봉천동,동_신림동,동_광장동,동_구의동,동_군자동,동_능동,동_자양동,동_중곡동,동_화양동,동_가리봉동,동_개봉동,동_고척동,동_구로동,동_궁동,동_신도림동,동_오류동,동_온수동,동_천왕동,동_항동,동_가산동,동_독산동,동_시흥동,동_공릉동,동_상계동,동_월계동,동_중계동,동_하계동,동_도봉동,동_방학동,동_쌍문동,동_창동,동_답십리동,동_신설동,동_용두동,동_이문동,동_장안동,동_전농동,동_제기동,동_청량리동,동_회기동,동_휘경동,동_노량진동,동_대방동,동_동작동,동_본동,동_사당동,동_상도1동,동_상도동,동_신대방동,동_흑석동,동_공덕동,동_구수동,동_노고산동,동_당인동,동_대흥동,동_도화동,동_동교동,동_마포동,동_망원동,동_상수동,동_상암동,동_서교동,동_성산동,동_신공덕동,동_신수동,동_신정동,동_아현동,동_연남동,동_염리동,동_용강동,동_중동,동_창전동,동_토정동,동_하중동,동_합정동,동_현석동,동_남가좌동,동_냉천동,동_대신동,동_대현동,동_미근동,동_봉원동,동_북가좌동,동_북아현동,동_신촌동,동_연희동,동_영천동,동_옥천동,동_창천동,동_천연동,동_충정로2가,동_충정로3가,동_합동,동_현저동,동_홍은동,동_홍제동,동_내곡동,동_반포동,동_방배동,동_서초동,동_신원동,동_양재동,동_염곡동,동_우면동,동_잠원동,동_금호동1가,동_금호동2가,동_금호동3가,동_금호동4가,동_도선동,동_마장동,동_사근동,동_상왕십리동,동_성수동1가,동_성수동2가,동_송정동,동_옥수동,동_용답동,동_응봉동,동_하왕십리동,동_행당동,동_홍익동,동_길음동,동_돈암동,동_동선동1가,동_동선동2가,동_동선동3가,동_동선동4가,동_동선동5가,동_동소문동1가,동_동소문동3가,동_동소문동4가,동_동소문동5가,동_동소문동6가,동_동소문동7가,동_보문동1가,동_보문동2가,동_보문동3가,동_보문동4가,동_보문동5가,동_보문동6가,동_보문동7가,동_삼선동1가,동_삼선동2가,동_삼선동3가,동_삼선동4가,동_삼선동5가,동_상월곡동,동_석관동,동_성북동,동_성북동1가,동_안암동1가,동_안암동2가,동_안암동3가,동_안암동4가,동_안암동5가,동_장위동,동_정릉동,동_종암동,동_하월곡동,동_가락동,동_거여동,동_마천동,동_문정동,동_방이동,동_삼전동,동_석촌동,동_송파동,동_신천동,동_오금동,동_잠실동,동_장지동,동_풍납동,동_목동,동_신월동,동_당산동,동_당산동1가,동_당산동2가,동_당산동3가,동_당산동4가,동_당산동5가,동_당산동6가,동_대림동,동_도림동,동_문래동1가,동_문래동2가,동_문래동3가,동_문래동4가,동_문래동5가,동_문래동6가,동_신길동,동_양평동1가,동_양평동2가,동_양평동3가,동_양평동4가,동_양평동5가,동_양평동6가,동_여의도동,동_영등포동,동_영등포동1가,동_영등포동2가,동_영등포동3가,동_영등포동4가,동_영등포동5가,동_영등포동6가,동_영등포동7가,동_영등포동8가,동_갈월동,동_남영동,동_도원동,동_동빙고동,동_동자동,동_문배동,동_보광동,동_산천동,동_서계동,동_서빙고동,동_신계동,동_신창동,동_용문동,동_용산동2가,동_용산동3가,동_용산동5가,동_원효로1가,동_원효로2가,동_원효로3가,동_원효로4가,동_이촌동,동_이태원동,동_주성동,동_청암동,동_청파동1가,동_청파동2가,동_청파동3가,동_한강로1가,동_한강로2가,동_한강로3가,동_한남동,동_효창동,동_후암동,동_갈현동,동_구산동,동_녹번동,동_대조동,동_불광동,동_수색동,동_역촌동,동_응암동,동_증산동,동_진관동,동_가회동,동_견지동,동_경운동,동_계동,동_관수동,동_관철동,동_교북동,동_구기동,동_낙원동,동_내수동,동_누상동,동_누하동,동_당주동,동_돈의동,동_동숭동,동_명륜1가,동_명륜2가,동_명륜3가,동_명륜4가,동_무악동,동_부암동,동_사직동,동_삼청동,동_수송동,동_숭인동,동_신교동,동_신문로1가,동_신문로2가,동_신영동,동_안국동,동_연건동,동_연지동,동_옥인동,동_운니동,동_원남동,동_원서동,동_이화동,동_익선동,동_인사동,동_인의동,동_적선동,동_종로1가,동_창성동,동_창신동,동_청운동,동_체부동,동_충신동,동_통의동,동_통인동,동_평동,동_평창동,동_필운동,동_행촌동,동_혜화동,동_홍지동,동_홍파동,동_화동,동_효자동,동_효제동,동_광희동1가,동_광희동2가,동_남대문로5가,동_남산동2가,동_남창동,동_만리동1가,동_만리동2가,동_무학동,동_묵정동,동_산림동,동_순화동,동_신당동,동_쌍림동,동_예장동,동_오장동,동_을지로2가,동_을지로3가,동_을지로4가,동_을지로5가,동_을지로6가,동_을지로7가,동_의주로1가,동_인현동2가,동_장충동1가,동_장충동2가,동_저동2가,동_정동,동_중림동,동_충무로2가,동_충무로3가,동_충무로4가,동_충무로5가,동_필동1가,동_필동2가,동_필동3가,동_황학동,동_회현동1가,동_회현동2가,동_흥인동,동_망우동,동_면목동,동_묵동,동_상봉동,동_신내동,동_중화동,일괄계약_N,일괄계약_Y
0,서울특별시 강남구 개포동 12 1992,1992,33.18,1,206607.181143,443586.061143,2,14,0.034422,0.999407,15900,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

# Sklearn Linear Regressor

In [3]:
from sklearn.linear_model import LinearRegression

In [4]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate

In [5]:
gkfold = GroupKFold(n_splits=5)

In [6]:
model = LinearRegression()

In [7]:
x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)'])
y_train = training_data_df['거래/보증금(만원)']

In [13]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [19]:
cv_results = cross_validate(
    model, x_train, y_train, cv=gkfold, groups=training_data_df['주소_건축년도'], n_jobs=5, verbose=1,
    return_train_score=True, return_estimator=True, scoring='neg_root_mean_squared_error'
)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:  4.8min remaining:  7.1min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:  4.9min finished


In [20]:
cv_results

{'fit_time': array([264.78313923, 118.99162436, 220.52851796, 264.57909369,
        264.36004353]),
 'score_time': array([1.57635188, 3.01767278, 6.04434705, 1.63636565, 1.59635639]),
 'estimator': [LinearRegression(),
  LinearRegression(),
  LinearRegression(),
  LinearRegression(),
  LinearRegression()],
 'test_score': array([-22634.75446428, -19850.55608454, -21569.23047192, -20840.79034226,
        -20319.11461539]),
 'train_score': array([-19878.49726457, -20567.04711361, -20201.92057127, -20358.86350675,
        -20442.88604148])}

In [6]:
#count = 0
models_list = []
for i, (train_idx, val_idx) in enumerate(gkfold.split(training_data_df, groups=training_data_df['주소_건축년도'])):
    print('-------------- Fold {} starting ---------------'.format(i))
    
    print(i)
    print(len(train_idx), len(val_idx))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]

    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]

    model = LinearRegression(n_jobs=-1)
    
    model.fit(x_train, y_train)
    
    preds = model.predict(x_val)
    
    score = mean_squared_error(y_val, preds)
    print(np.sqrt(score))
    
    models_list.append(model)
    
    print('-------------- Fold {} ending ---------------'.format(i))
    #count += 1

-------------- Fold 0 starting ---------------
0
1673207 418302
22634.754464278878
-------------- Fold 0 ending ---------------
-------------- Fold 1 starting ---------------
1
1673207 418302
19850.556084539403
-------------- Fold 1 ending ---------------
-------------- Fold 2 starting ---------------
2
1673207 418302
21569.230471922627
-------------- Fold 2 ending ---------------
-------------- Fold 3 starting ---------------
3
1673207 418302
20840.79034225961
-------------- Fold 3 ending ---------------
-------------- Fold 4 starting ---------------
4
1673208 418301
20319.114615388153
-------------- Fold 4 ending ---------------


In [8]:
for model in models_list:
    print(model)

LinearRegression(n_jobs=-1)
LinearRegression(n_jobs=-1)
LinearRegression(n_jobs=-1)
LinearRegression(n_jobs=-1)
LinearRegression(n_jobs=-1)


In [9]:
from joblib import dump, load

In [11]:
for i, model in enumerate(models_list):
    dump(model, './Training/Training7_20220208/linear_regression_model_{}.joblib'.format(i))

# Sklearn Lasso

In [21]:
from sklearn.linear_model import Lasso

In [22]:
gkfold = GroupKFold(n_splits=5)

In [23]:
#count = 0
models_list = []
for i, (train_idx, val_idx) in enumerate(gkfold.split(training_data_df, groups=training_data_df['주소_건축년도'])):
    print('-------------- Fold {} starting ---------------'.format(i))
    
    print(i)
    print(len(train_idx), len(val_idx))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]

    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]

    model = Lasso()
    
    model.fit(x_train, y_train)
    
    preds = model.predict(x_val)
    
    score = mean_squared_error(y_val, preds)
    print(np.sqrt(score))
    
    models_list.append(model)
    
    print('-------------- Fold {} ending ---------------'.format(i))
    #count += 1

-------------- Fold 0 starting ---------------
0
1673207 418302


c:\users\max6296\ml\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.343e+14, tolerance: 2.393e+11
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


22629.132559956903
-------------- Fold 0 ending ---------------
-------------- Fold 1 starting ---------------
1
1673207 418302


c:\users\max6296\ml\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.579e+14, tolerance: 2.451e+11
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


19820.480003389595
-------------- Fold 1 ending ---------------
-------------- Fold 2 starting ---------------
2
1673207 418302


...

21566.46730578575
-------------- Fold 2 ending ---------------
-------------- Fold 3 starting ---------------
3
1673207 418302


c:\users\max6296\ml\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.511e+14, tolerance: 2.425e+11
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


20788.660291698146
-------------- Fold 3 ending ---------------
-------------- Fold 4 starting ---------------
4
1673208 418301


c:\users\max6296\ml\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.539e+14, tolerance: 2.457e+11
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


20296.07078898276
-------------- Fold 4 ending ---------------


In [24]:
from sklearn.linear_model import LassoCV

In [25]:
x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)'])
y_train = training_data_df['거래/보증금(만원)']

In [27]:
model = LassoCV(
    cv=gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']),
    n_jobs=5,
    verbose=1,
)

In [28]:
model.fit(x_train, y_train)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
........................................................................................................................................................................................................[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:  1.7min remaining:  2.5min
............................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:  2.5min finished


LassoCV(cv=<generator object _BaseKFold.split at 0x00000276C29E92C8>, n_jobs=5,
        verbose=1)

In [30]:
model.score

TypeError: score() missing 2 required positional arguments: 'X' and 'y'

# Sklearn Quantile Regressor

In [12]:
from sklearn.linear_model import QuantileRegressor

In [13]:
#count = 0
models_list = []
for i, (train_idx, val_idx) in enumerate(gkfold.split(training_data_df, groups=training_data_df['주소_건축년도'])):
    print('-------------- Fold {} starting ---------------'.format(i))
    
    print(i)
    print(len(train_idx), len(val_idx))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]

    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]

    model = QuantileRegressor(quantile=0.5)
    
    model.fit(x_train, y_train)

    dump(model, './Training/Training7_20220208/quantile05_regression_model_{}.joblib'.format(i))
    
    print('-------------- Fold {} ending ---------------'.format(i))
    #count += 1

-------------- Fold 0 starting ---------------
0
1673207 418302


MemoryError: Unable to allocate 20.4 TiB for an array with shape (1673207, 1673207) and data type float64

In [ ]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    model = QuantileRegressor(quantile=0.25)
    
    model.fit(x_train, y_train)

    dump(model, './Training/Training7_20220208/quantile05_regression_model_{}.joblib'.format(i))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

In [ ]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.75')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training6_20220207/catboost_quantile075_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

In [ ]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.125')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training6_20220207/catboost_quantile0125_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1

In [ ]:
count = 0
models_list = []
for train_idx, val_idx in gkfold.split(training_data_df, groups=training_data_df['주소_건축년도']):
    print('-------------- Fold {} starting ---------------'.format(count))
    
    x_train = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[train_idx, :]
    y_train = training_data_df['거래/보증금(만원)'].loc[train_idx]
    
    x_val = training_data_df.drop(columns=['주소_건축년도', '거래/보증금(만원)']).loc[val_idx, :]
    y_val = training_data_df['거래/보증금(만원)'].loc[val_idx]
    
    train_pool = catboost.Pool(x_train, y_train, cat_features=cat_cols)
    val_pool = catboost.Pool(x_val, y_val, cat_features=cat_cols)
    
    model = catboost.CatBoostRegressor(iterations=10000, depth=6, task_type='GPU', loss_function='Quantile:alpha=0.875')
    
    model.fit(train_pool, eval_set=val_pool, verbose=50, early_stopping_rounds=200)
    
    model.save_model('./Training/Training6_20220207/catboost_quantile0875_{}'.format(count))
    
    print('-------------- Fold {} ending ---------------'.format(count))
    count += 1